<a href="https://colab.research.google.com/github/OrangeLyx/507final_369/blob/main/train_part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hi, it is a part of training code including: Logistic Regression, SVM, Naive Baysien, DistilBERT and deBERTa.

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import string
from sklearn.metrics import accuracy_score, confusion_matrix
!pip install datasets
from datasets import load_dataset,Dataset
from sklearn.model_selection import train_test_split

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
imdb = load_dataset("imdb")
print(imdb)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


You can try upload your data from this device. We directly use the data from HuggingFace library.

In [ ]:
# upload from the device
with open("./IMDB Dataset.csv", "r", encoding="utf-8") as f:
    data = pd.read_csv(f)

ParserError: Error tokenizing data. C error: EOF inside string starting at row 34067

In [ ]:
# data pre-processing
train_dataset=imdb["train"].shuffle(seed=123)
test_dataset=imdb["test"].shuffle(seed=123)

data_train= pd.DataFrame({"text": train_dataset["text"], "label":train_dataset["label"]})
data_test= pd.DataFrame({"text": test_dataset["text"], "label":test_dataset["label"]})

def clean_text(text):
    text=text.lower()
    text=re.sub('\[.*?\]','',text)
    text=re.sub(r'<.*?>','',text)
    text=re.sub('[%s]'%re.escape(string.punctuation),'',text)
    text=re.sub(r'[^a-zA-Z0-9\s]','',text)
    text=re.sub('[''"",,,]','',text)
    text=re.sub('\n','',text)
    stop_words = {'a', 'an', 'the'}
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

cleaned=lambda x:clean_text(x)

data_test['text']=pd.DataFrame(data_test.text.apply(cleaned))
data_train['text']=pd.DataFrame(data_train.text.apply(cleaned))


data_test.head(10)
data_train.head(10)

,text,label
0,i wont claim to be fan of ralph bakshi because...,1
1,i was not impressed about this film especially...,0
2,aka demons iiimade for italian tv although sho...,0
3,thief of baghdad is one of my ten alltime favo...,1
4,this is first 10 out of 10 that ive given any ...,1
5,there is uk edition to this show which is rath...,0
6,spoilersi saw original on tv sometime ago and ...,1
7,spoilers powerhouse movie that shows how men i...,1
8,this movie is intelligent that is more than mo...,1
9,despite unfortunately thinking itself to be in...,0


In [ ]:
data_all = pd.concat([data_train, data_test], axis=0).reset_index(drop=True)
data_train, data_test= train_test_split(data_all, test_size=0.25, random_state=42, stratify=data_all["label"])

print("Size of train dataset:", len(data_train))
print("Size of test dataset:", len(data_test))
print(data_train["label"].value_counts(normalize=True))
print(data_test["label"].value_counts(normalize=True))

dataset_train = Dataset.from_pandas(data_train)
dataset_test = Dataset.from_pandas(data_test)

训练集大小: 37500
验证集大小: 12500
label
0    0.5
1    0.5
Name: proportion, dtype: float64
label
0    0.5
1    0.5
Name: proportion, dtype: float64


Here is a few EDA steps for this project. We create a word cloud to check the frequency of this corpus from dataset.

In [ ]:
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:

# statstics analysis for word cloud
text_label_1 = " ".join(data_train[data_train['label'] == 1]['text'])
text_label_0 = " ".join(data_train[data_train['label'] == 0]['text'])


remove_words = ["movie", "film"]

def preprocess_text_for_wordcloud(text, remove_words=None):
    vectorizer = CountVectorizer(stop_words='english')
    X = vectorizer.fit_transform([text])
    word_counts = pd.DataFrame({'word': vectorizer.get_feature_names_out(), 'count': X.toarray().sum(axis=0)})

    if remove_words:
        word_counts = word_counts[~word_counts['word'].isin(remove_words)]

    return dict(zip(word_counts['word'], word_counts['count']))

word_freq_1 = preprocess_text_for_wordcloud(text_label_1, remove_words)
word_freq_0 = preprocess_text_for_wordcloud(text_label_0, remove_words)


In [ ]:

def generate_wordcloud(word_freq, title, save_path):
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(word_freq)
    plt.figure(figsize=(10, 6))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.savefig(save_path, format='png', dpi=300)
    plt.close()

# graph
generate_wordcloud(word_freq_0, "Word Cloud for Positive Text in train data","./wordcloud_positive.png")
generate_wordcloud(word_freq_1, "Word Cloud for Negative Text in train data","./wordcloud_negative.png")



In [ ]:

X_train=data_train['text']
y_train=data_train['label']
X_test=data_test['text']
y_test=data_test['label']


print("Training data shape:", X_train.shape, y_train.shape)
print("Testing data shape:", X_test.shape, y_test.shape)

Training data shape: (37500,) (37500,)
Testing data shape: (12500,) (12500,)


Logistics Regression Classifer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report,precision_score, recall_score, f1_score

In [ ]:
tf = TfidfVectorizer()

X_train_tf = tf.fit_transform(X_train)
X_test_tf= tf.transform(X_test)

In [ ]:
model = LogisticRegression(max_iter=100)
model.fit(X_train_tf, y_train)

y_pred = model.predict(X_test_tf)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print("Classification Report:\n", report)

Accuracy: 0.89592
Precision: 0.8961
Recall: 0.8959
F1-Score: 0.8959
Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.88      0.89      6250
           1       0.89      0.91      0.90      6250

    accuracy                           0.90     12500
   macro avg       0.90      0.90      0.90     12500
weighted avg       0.90      0.90      0.90     12500



Support Vector Machine Classifier

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report,precision_score, recall_score, f1_score

In [ ]:
tf = TfidfVectorizer()

X_train_tf = tf.fit_transform(X_train)
X_test_tf= tf.transform(X_test)

model = SVC(kernel='linear')
model.fit(X_train_tf, y_train)

y_pred = model.predict(X_test_tf)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print("Classification Report:\n", report)

Accuracy: 0.89904
Precision: 0.8991
Recall: 0.8990
F1-Score: 0.8990
Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.89      0.90      6250
           1       0.89      0.91      0.90      6250

    accuracy                           0.90     12500
   macro avg       0.90      0.90      0.90     12500
weighted avg       0.90      0.90      0.90     12500



Naive Baysien Classifier

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report,precision_score, recall_score, f1_score

tf = TfidfVectorizer()

X_train_tf = tf.fit_transform(X_train)
X_test_tf = tf.transform(X_test)

model = MultinomialNB(alpha=0.6)  # 多项式朴素贝叶斯
model.fit(X_train_tf, y_train)

y_pred = model.predict(X_test_tf)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print("Classification Report:\n", report)


Accuracy: 0.8672
Precision: 0.8678
Recall: 0.8672
F1-Score: 0.8672
Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.89      0.87      6250
           1       0.88      0.85      0.86      6250

    accuracy                           0.87     12500
   macro avg       0.87      0.87      0.87     12500
weighted avg       0.87      0.87      0.87     12500



DistilBERT fine-tuning

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
!pip install datasets
from datasets import Dataset

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True)

dataset_train = Dataset.from_pandas(data_train)
dataset_test = Dataset.from_pandas(data_test)


tokenized_train = dataset_train .map(preprocess_function, batched=True)
tokenized_test = dataset_test.map(preprocess_function, batched=True)

Map:   0%|          | 0/37500 [00:00<?, ? examples/s]

Map:   0%|          | 0/12500 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
import numpy as np
!pip install evaluate
from evaluate import load


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)

    acc = accuracy_score(labels, predictions)

    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average="binary"
    )

    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./output_distilbert",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

<ipython-input-39-03c0eb7b9a35>:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Step,Training Loss
500,0.296600
1000,0.267800
1500,0.238000
2000,0.227700
2500,0.194700
3000,0.145800
3500,0.161700
4000,0.152900
4500,0.147000
5000,0.108400


TrainOutput(global_step=7032, training_loss=0.16295439108307722, metrics={'train_runtime': 1179.0961, 'train_samples_per_second': 95.412, 'train_steps_per_second': 5.964, 'total_flos': 1.49025823488e+16, 'train_loss': 0.16295439108307722, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.292978435754776,
 'eval_accuracy': 0.93112,
 'eval_precision': 0.9318801089918256,
 'eval_recall': 0.93024,
 'eval_f1': 0.9310593322123468,
 'eval_runtime': 46.0997,
 'eval_samples_per_second': 271.152,
 'eval_steps_per_second': 16.963,
 'epoch': 3.0}

In [ ]:
output_dir = "./trained_distilbert"
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

DeBERTa-v3-base fine-tuning

In [ ]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification,TrainingArguments,Trainer

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)


tokenized_train = dataset_train.map(preprocess_function, batched=True)
tokenized_test = dataset_test.map(preprocess_function, batched=True)

# Remove unnecessary original columns, keeping only model inputs and labels
tokenized_train = tokenized_train.remove_columns(["text"])
tokenized_test = tokenized_test.remove_columns(["text"])


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [ ]:

model = AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-v3-base", num_labels=2)
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:


training_args = TrainingArguments(
    output_dir="./results_deberta",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    save_total_limit=1,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.250100,0.248383,0.903480,0.909439
2,0.174200,0.245158,0.922040,0.921243
3,0.114000,0.319526,0.923640,0.923965


TrainOutput(global_step=4689, training_loss=0.18752569164846417, metrics={'train_runtime': 2997.0292, 'train_samples_per_second': 25.025, 'train_steps_per_second': 1.565, 'total_flos': 4933420761600000.0, 'train_loss': 0.18752569164846417, 'epoch': 3.0})

In [ ]:

trainer.evaluate()

{'eval_loss': 0.3195260465145111,
 'eval_accuracy': 0.92364,
 'eval_f1': 0.9239654279682957,
 'eval_runtime': 238.8435,
 'eval_samples_per_second': 104.671,
 'eval_steps_per_second': 6.544,
 'epoch': 3.0}

In [ ]:
trainer.save_model("./fine_tuned_deberta_v3_base")
tokenizer.save_pretrained("./fine_tuned_deberta_v3_base")